In [1]:
!pip -q install pypdf scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.4 MB/s eta 0:00:00


In [2]:
import re, json, pickle, os
from pathlib import Path
from typing import List, Dict, Tuple
from pypdf import PdfReader

In [3]:
from google.colab import files

print("👉 Click 'Choose Files' and select your PDF (e.g., Crawfords_Auto_Repair_Guide.pdf)")
uploaded = files.upload()
assert len(uploaded) > 0, "No file uploaded."
pdf_path = list(uploaded.keys())[0]
print("Uploaded:", pdf_path)

👉 Click 'Choose Files' and select your PDF (e.g., Crawfords_Auto_Repair_Guide.pdf)


Saving Crawfords_Auto_Repair_Guide.pdf to Crawfords_Auto_Repair_Guide.pdf
Uploaded: Crawfords_Auto_Repair_Guide.pdf


In [4]:
def clean_text(t: str) -> str:
    t = (t or "").replace("\r", " ")
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\n{3,}", "\n\n", t)
    return t.strip()

reader = PdfReader(pdf_path)
pages = []
extract_errors = []

for i, page in enumerate(reader.pages):
    try:
        txt = clean_text(page.extract_text())
    except Exception as e:
        txt = ""
        extract_errors.append((i, repr(e)))
    pages.append({"page": i+1, "text": txt})

print(f"Pages extracted: {len(pages)}")
if extract_errors:
    print("Some pages had extraction issues (showing a few):", extract_errors[:5])


Pages extracted: 84


In [5]:
base_dir = Path("car_manual_raw")
base_dir.mkdir(exist_ok=True)

# Save concatenated text with page markers
with open(base_dir/"manual.txt", "w", encoding="utf-8") as f:
    for p in pages:
        f.write(f"[PAGE {p['page']}]\n{p['text']}\n\n")

# Save JSONL page-by-page
with open(base_dir/"pages.jsonl", "w", encoding="utf-8") as f:
    for p in pages:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")

print("Saved to:", base_dir.resolve())


Saved to: /content/car_manual_raw


In [6]:
# Show the first ~1200 characters to confirm extraction looks okay
all_text_preview = "\n\n".join([f"[PAGE {p['page']}]\n{p['text']}" for p in pages])[:1200]
print(all_text_preview)

# Simple keyword search to see where topics live
def find_pages(keyword: str, max_hits: int = 20):
    hits = []
    kw = keyword.lower()
    for p in pages:
        if kw in p["text"].lower():
            hits.append(p["page"])
            if len(hits) >= max_hits:
                break
    return hits

for kw in ["tire", "tire pressure", "coolant", "brake", "battery"]:
    print(f"{kw!r} -> pages:", find_pages(kw))


[PAGE 1]
TThhee
Guide to 
Beginners Auto 
Maintenance & Repair 
Jeff Crawford

[PAGE 2]
Presented by: 
About the Author: Jeff Crawford has over 20 years of experience as a mechanic and 
auto technician. He has seen all aspects of the auto repair industry from small shops to 
large dealerships. He is the co-owner of Crawfords Auto Repair in Mesa, Arizona 
85210. www.CrawfordsAutoService.com .

[PAGE 3]
The Crawfords Auto Repair Guide to Beginners Auto Maintenance & Repair 
By Jeff Crawford, Crawfords Auto Repair 
Contributing Author: Rex Kimball, Mirex Marketing 
Text copyright © Jeff Crawford, Rex Kimball 
All rights reserved for printed and all digital formats. The following license is granted in 
PDF: 
This book is available in PDF for free, and it can be reproduced, shared and sold for 
profit royalty-free in PDF, but it a cannot be changed or adapted in content or format 
without the authors consent. Go go http://www.crawfordsautoservice.com/crawfords- 
auto-repair-guide-free